In [1]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from keras.layers import Input, Dense
from keras.models import Layer, Model

from keras.optimizers import SGD, Nadam

import keras.backend as K
import numpy as np
seed=6
np.random.seed(seed)

Using Theano backend.


In [2]:
aa_length=50
nb_filter=20

In [3]:
dense = Dense(output_dim=nb_filter, init='glorot_uniform', activation='sigmoid')

In [4]:
from keras import activations, initializations, regularizers, constraints
from keras.engine import InputSpec
from keras.utils.np_utils import conv_output_length


class Dedense(Layer):

    def __init__(self, bound_dense_layer, init='glorot_uniform', activation='linear', weights=None,
                 W_regularizer=None, b_regularizer=None, activity_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, input_dim=None, **kwargs):
        self.init = initializations.get(init)
        self.activation = activations.get(activation)
        
        self._bound_dense_layer = bound_dense_layer
        
        try:
            self.output_dim = self._bound_conv_layer.input_shape[0]
        except Exception:
            self.output_dim = 'Not sure yet, input shape of dense layer not provided during construction.'        
        
        self.input_dim = input_dim

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.initial_weights = weights
        self.input_spec = [InputSpec(ndim=2)]

        if self.input_dim:
            kwargs['input_shape'] = (self.input_dim,)
        super(Dedense, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = [InputSpec(dtype=K.floatx(),
                                     shape=(None, input_dim))]
        
        self.output_dim = self._bound_dense_layer.input_shape[1]
        self.W = self._bound_dense_layer.W.T
        
        if self.bias:
            self.b = K.zeros((self.output_dim,),
                             name='{}_b'.format(self.name))
            self.trainable_weights = [self.b]
        else:
            self.trainable_weights = []

        self.regularizers = []
        if self.W_regularizer:
            self.W_regularizer.set_param(self.W)
            self.regularizers.append(self.W_regularizer)

        if self.bias and self.b_regularizer:
            self.b_regularizer.set_param(self.b)
            self.regularizers.append(self.b_regularizer)

        if self.activity_regularizer:
            self.activity_regularizer.set_layer(self)
            self.regularizers.append(self.activity_regularizer)

        self.constraints = {}
        if self.W_constraint:
            self.constraints[self.W] = self.W_constraint
        if self.bias and self.b_constraint:
            self.constraints[self.b] = self.b_constraint

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        output = K.dot(x, self.W)
        if self.bias:
            output += self.b
        return self.activation(output)

    def get_output_shape_for(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return (input_shape[0], self.output_dim)

    def get_config(self):
        config = {'output_dim': self.output_dim,
                  'init': self.init.__name__,
                  'activation': self.activation.__name__,
                  'W_regularizer': self.W_regularizer.get_config() if self.W_regularizer else None,
                  'b_regularizer': self.b_regularizer.get_config() if self.b_regularizer else None,
                  'activity_regularizer': self.activity_regularizer.get_config() if self.activity_regularizer else None,
                  'W_constraint': self.W_constraint.get_config() if self.W_constraint else None,
                  'b_constraint': self.b_constraint.get_config() if self.b_constraint else None,
                  'bias': self.bias,
                  'input_dim': self.input_dim}
        base_config = super(Dedense, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [5]:
dedense = Dedense(dense)

In [6]:
inp = Input(shape=(aa_length,), name='aa_seq')
x_data = np.random.random((10, aa_length))

In [7]:
x_e = dense(inp)
x_d = dedense(x_e)

In [8]:
coder = Model(input=inp, output=x_d)

In [9]:
coder.count_params()

1070

In [20]:
coder.compile(optimizer=Nadam(lr=0.001), loss='mse')

In [21]:
coder.fit(x_data, x_data, nb_epoch=100)

Epoch 1/100
10/10 [==============================] - 0s - loss: 0.0010
Epoch 2/100
10/10 [==============================] - 0s - loss: 5.0677e-04
Epoch 3/100
10/10 [==============================] - 0s - loss: 3.5016e-04
Epoch 4/100
10/10 [==============================] - 0s - loss: 2.8031e-04
Epoch 5/100
10/10 [==============================] - 0s - loss: 2.4880e-04
Epoch 6/100
10/10 [==============================] - 0s - loss: 2.3678e-04
Epoch 7/100
10/10 [==============================] - 0s - loss: 2.3314e-04
Epoch 8/100
10/10 [==============================] - 0s - loss: 2.3240e-04
Epoch 9/100
10/10 [==============================] - 0s - loss: 2.3198e-04
Epoch 10/100
10/10 [==============================] - 0s - loss: 2.3169e-04
Epoch 11/100
10/10 [==============================] - 0s - loss: 2.3179e-04
Epoch 12/100
10/10 [==============================] - 0s - loss: 2.3327e-04
Epoch 13/100
10/10 [==============================] - 0s - loss: 2.3587e-04
Epoch 14/100
10/10 [=====

In [22]:
print(x_data.squeeze()[0])
print(coder.predict(x_data).squeeze()[0])
print(coder.evaluate(x_data, x_data))

[ 0.89286015  0.33197981  0.82122912  0.04169663  0.10765668  0.59505206
  0.52981736  0.41880743  0.33540785  0.62251943  0.43814143  0.73588211
  0.51803641  0.5788586   0.6453551   0.99022427  0.8198582   0.41320093
  0.87626766  0.82375943  0.05447451  0.71863724  0.80217056  0.73640664
  0.70913175  0.54093685  0.12482417  0.9576473   0.4032563   0.21695116
  0.71727585  0.99420744  0.25561406  0.67130943  0.59900591  0.71733215
  0.93734953  0.35180977  0.2536341   0.40247251  0.74651072  0.72407057
  0.4061078   0.98937985  0.45049928  0.37380843  0.70962861  0.08245855
  0.39837292  0.77088097]
[ 0.88782948  0.30225784  0.80226415  0.04064508  0.1059496   0.59051514
  0.52358276  0.39128494  0.32890975  0.61118323  0.42572653  0.72776437
  0.49785265  0.57518721  0.64433932  0.98385829  0.80992067  0.38780922
  0.83627772  0.80940825  0.04999152  0.70816571  0.77912211  0.73862946
  0.71417922  0.52391803  0.11557958  0.93964916  0.37638783  0.20576182
  0.69508624  0.95463437 

In [23]:
np.mean(np.square(x_data-coder.predict(x_data)), dtype=np.float32)

0.00014934808